# **Actorle Cheat Sheet**
### [Actorle](https://actorle.com/) is a fantastic game that is like wordle for actors where you have to guess the actor given their filmography. However, the titles of their films are blanked out. Instead, you have access to the films' genres and IMDB scores. If you guess the wrong actor but they share a film with the right one, this film will be revealed. Can we use data science to win at this game?

### Here we will find the most popular actor, the most popular actor by genre and then the actor who has appeared in the most films shared with other popular actors, essentially the ['Kevin Bacon'](https://simple.wikipedia.org/wiki/Bacon_number#:~:text=The%20Bacon%20number%20of%20an,concept%20to%20the%20movie%20industry.) of popular movie stars. The first two are obviously good for the game actorle but this last class is also important because guessing an actor who shares a lot of films with the correct answer will reveal the names of the films they share (as per the rules) providing useful information.

### The dataset we will be using can be found [here](https://www.kaggle.com/datasets/juzershakir/tmdb-movies-dataset?rvi=1).

## **1) Most popular actor (regardless of genre)**

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('tmdb_movies_data.csv')

In [3]:
#clean data
data = data[data["cast"].isnull() == False]
data = data[data["genres"].isnull() == False]

data = data[data.budget_adj != 0]
data = data[data.revenue_adj != 0]

In [10]:
#subset of data relevant for actorle
actorle = data[['original_title','popularity', 'genres','cast', 'vote_average']]

actorle.head(10)

'Tomorrowland'

In [5]:
actors = actorle['cast']
actors = actors.str.split('|')
freq = {}
for i in actors:
    for j in i:
        if j in freq:
            freq[j]+=1
        else:
            freq[j]=1
freq = sorted(freq.items(), key=lambda x:x[1], reverse = True)
print('The twenty most popular actors are')
print()
for i,j in enumerate(freq[:20]):
    print(i+1,j[0])



The twenty most popular actors are

1 Robert De Niro
2 Bruce Willis
3 Samuel L. Jackson
4 Nicolas Cage
5 Matt Damon
6 Johnny Depp
7 Harrison Ford
8 Brad Pitt
9 Tom Hanks
10 Sylvester Stallone
11 Morgan Freeman
12 Tom Cruise
13 Denzel Washington
14 Eddie Murphy
15 Liam Neeson
16 Owen Wilson
17 Julianne Moore
18 Arnold Schwarzenegger
19 Mark Wahlberg
20 Meryl Streep


### It is clear from the data above that a really good guess would be something like Robert De Niro, Bruce Willis or Samuel L. Jackson. Of course, we have not taken into account genre which is a crucial bit of information actorle gives us. We do that next.

# **2) Most popular actor by genre**

In [6]:
genres = actorle['genres']
genres = genres.str.split('|')

# Making a list of all genres with index keys
genre_list = []
for i in genres:
    for j in i:
        if j not in genre_list:
            genre_list.append(j)

print('List of genres:')
print(genre_list)
print()
genre_list_keys = {}
for i,j in enumerate(genre_list):
    genre_list_keys[j] = i


# Preparing a dictionary with entries: {... actor: (genre_list[0].freq, genre_list[1].freq...)...)}
actor_genre_freq = {}
for i in actors:
    for j in i:
        if j not in actor_genre_freq:
            actor_genre_freq[j] = [0]*len(genre_list)        
# Filling dictionary
for i,j in enumerate(actors):
    for actor in j:
        try:
            for genre in genres[i]:
                actor_genre_freq[actor][genre_list_keys[genre]]+=1
        except KeyError:
            pass
            
# Now one can find greatest appearances per genre, e.g. action       
genre = 'Documentary'
actor_genre_freq = sorted(actor_genre_freq.items(), key=lambda x:x[1][genre_list_keys[genre]], reverse = True)  

print('The twenty most popular actors in the genre of %s are:' %genre)
print()
for i,j in enumerate(actor_genre_freq[:20]):
    print(i+1,j[0])

List of genres:
['Action', 'Adventure', 'Science Fiction', 'Thriller', 'Fantasy', 'Crime', 'Western', 'Drama', 'Family', 'Animation', 'Comedy', 'Mystery', 'Romance', 'War', 'History', 'Music', 'Horror', 'Documentary', 'Foreign', 'TV Movie']

The twenty most popular actors in the genre of Documentary are:

1 Adam Sandler
2 Kevin James
3 Rachel McAdams
4 Ellen Burstyn
5 Cillian Murphy
6 Selena Gomez
7 Steve Buscemi
8 Djimon Hounsou
9 Edward Norton
10 Evan Rachel Wood
11 Danny Glover
12 Linda Blair
13 Max von Sydow
14 Jayma Mays
15 Ned Beatty
16 Chris Brown
17 David Morse
18 Robert Carlyle
19 Bruce Dern
20 David Spade


### If one checks out 'Action', 'Adventure', 'Drama', 'Crime' Samuel. L. Jackson is near or at the top of all of them. He's looking like our best bet

## **Aside: list of movies for a given actor**

In [18]:
name_of_actor = 'Laura Dern'
list_of_films = []

for i,j in enumerate(actors):
    for actor in j:
        if actor == name_of_actor:
            try:
                list_of_films.append(actorle.loc[i, 'original_title'])
            except KeyError:
                pass
print('%s has been in these films:' %name_of_actor)
print()
print(list_of_films)

Laura Dern has been in these films:

['The Interview', 'The Iron Lady']


## **Aside aside: list of actors in a given film**

In [27]:
name_of_film = 'The Dark Knight'
actor_list = actorle.loc[actorle['original_title'] == name_of_film,'cast']
print('%s has the following top-billed cast:' %name_of_film)
print()
print(list(actor_list.str.split('|')))

The Dark Knight has the following top-billed cast:

[['Christian Bale', 'Michael Caine', 'Heath Ledger', 'Aaron Eckhart', 'Gary Oldman']]
